# Installs

In [9]:
!pip install transformers sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached

In [ ]:
!pip install lightning

In [ ]:
# !pip install --force-reinstall --no-deps git+https://github.com/tony-pitchblack/nltk/

In [ ]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

# Data prep

## Corpus loading

In [ ]:
!pip install corus wget

In [10]:
import wget

url_lenta = 'https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz'
url_ria = 'https://github.com/RossiyaSegodnya/ria_news_dataset/raw/master/ria.json.gz'

filename = wget.download(url_lenta, )

In [11]:
# from corus import load_ria
# corpus_path = 'data/ria.json.gz'
from corus import load_lenta
corpus_path = './lenta-ru-news.csv.gz'

load_func = load_lenta
records = load_func(corpus_path)
skip = 2
for i in range(skip):
    rec = next(records)
print(rec.title)
print(rec.text)

Австрия не представила доказательств вины российских биатлонистов
Австрийские правоохранительные органы не представили доказательств нарушения российскими биатлонистами антидопинговых правил. Об этом сообщил посол России в Вене Дмитрий Любинский по итогам встречи уполномоченного адвоката дипмиссии с представителями прокуратуры страны, передает ТАСС. «Действует презумпция невиновности. Каких-либо ограничений свободы передвижения для команды нет», — добавили в посольстве. Международный союз биатлонистов (IBU) также не будет применять санкции к российским биатлонистам. Все они продолжат выступление на Кубке мира. Полиция нагрянула в отель сборной России в Хохфильцене вечером 12 декабря. Как написал биатлонист Александр Логинов, их считают виновными в махинациях с переливанием крови. Биатлонисту Антону Шипулину, также попавшему в список, полиция нанесла отдельный визит: сейчас он тренируется отдельно в австрийском Обертиллахе. Обвинения спортсмен назвал бредом, а также указал на «охоту на 

In [16]:
from torch.utils.data import IterableDataset, DataLoader
import subprocess
import wget
import os
import nltk
nltk.download('punkt')

class TextSegDataset(IterableDataset):
    def __init__(self, load_func, corpus_url, articles_per_doc, ret_titles=False):
        self.corpus_path = os.path.basename(corpus_url) # wget saves to current dir
        self.sentence_tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')

        # save args
        self.__dict__.update(locals())
        del self.__dict__["self"]

        !wget -nc {corpus_url}

    def generate_doc(self):
        records = self.load_func(self.corpus_path)

        # corpus = ''
        titles = []
        true_seg_bounds = []
        doc_sentences = []

        total_records = self.articles_per_doc

        is_last_record = False
        while not is_last_record:
            rec = next(records)
            article_count = 1
            while True:
                section_sentences = self.sentence_tokenizer.tokenize(rec.text)

                doc_sentences.extend(section_sentences)
                if self.ret_titles:
                    titles.append(rec.title)

                # update true segment bounds
                for _ in range(len(section_sentences)-1):
                    true_seg_bounds.append(0)

                article_count += 1
                # if current record is not last
                try:
                    rec = next(records)
                    if (article_count + 1) % self.articles_per_doc == 0:
                        output = [
                            doc_sentences,
                            true_seg_bounds
                        ]

                        if self.ret_titles:
                            output.append(titles)

                        titles = []
                        true_seg_bounds = []
                        doc_sentences = []
                        break
                    else:
                        # corpus += '\n\n'
                        true_seg_bounds.append(1)
                except StopIteration:
                    is_last_record = True
                    break

            yield output


    def configure_iterator(self, ret_titles):
        self.ret_titles = ret_titles

    def __iter__(self):
        doc_generator = self.generate_doc()
        return iter(doc_generator)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [17]:
dataset = TextSegDataset(load_lenta, url_lenta, 10)

File ‘lenta-ru-news.csv.gz’ already there; not retrieving.



In [18]:
def collate_fn(data):
    return data

In [19]:
dl = DataLoader(dataset, batch_size=3, collate_fn=collate_fn)

In [20]:
res = next(iter(dl))
len(res)
i = 0
print(len(res[i][0]), len(res[i][1]))
# TODO: fix true seg bounds lengths
for r in res:
    print(r[:10])

96 95
[['Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости.', 'По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе.', 'Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения.', 'В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года.', 'По данным Росстата, в 2017 году от рака умерли 289 тысяч человек.', 'Это на 3,5 процента меньше, чем годом ранее.', 'Австрийские правоохранительные органы не представили доказательств нарушения российскими биатлонистами антидопинговых правил.', 'Об этом сообщил посол России в Вене Дмитрий Любинский по итогам встречи уполномоченного адвоката дипмиссии с представителями прокуратуры страны, передает ТАСС.', '«Действует презумпци

In [21]:
class TestIterableDataset(IterableDataset):
    def __init__(self, data, max_doc_length):
        self.data = data
        self.max_doc_length = max_doc_length

    def generator(self):
        # for i, d in enumerate(self.data):
        #     yield d
        new_doc = []
        for i, d in enumerate(self.data):
            new_doc.append(d)
            if (i+1) % self.max_doc_length == 0:
                doc = new_doc
                new_doc = []
                yield doc

    def __iter__(self):
        gen = self.generator()
        return iter(gen)


In [22]:
import numpy as np
from torch.utils.data import DataLoader

data = np.arange(12)*8
print(data)
it_dataset = TestIterableDataset(data, 2)
loader = DataLoader(it_dataset, batch_size=3, collate_fn=collate_fn)

for seg in iter(it_dataset):
    print(seg)

for batch in loader:
    print(batch)

[ 0  8 16 24 32 40 48 56 64 72 80 88]
[0, 8]
[16, 24]
[32, 40]
[48, 56]
[64, 72]
[80, 88]
[[0, 8], [16, 24], [32, 40]]
[[48, 56], [64, 72], [80, 88]]


# Model

In [65]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence as PACK, \
    pad_packed_sequence as PAD
# from torch.nn.utils.rnn import pad_sequence as PAD
import lightning as L
from abc import ABC, abstractmethod

class LSTMNetBase(nn.Module, ABC):
    def __init__(self, tokenizer, nse_model,
                hidden_dim, num_layers, output_dim=1, dropout=0, bidirectional=True):
        super().__init__()
        self.bidirectional = bidirectional

        self.tokenizer = tokenizer
        self.nse_model = nse_model
        embedding_dim = nse_model.config.hidden_size

        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_dim,
            num_layers=num_layers,
            bidirectional=bidirectional,
            dropout=dropout,
            batch_first=True
        )
        if bidirectional:
            linear_in_size = hidden_dim * 2
        else:
            linear_in_size = hidden_dim

        self.non_linear = nn.Tanh()
        self.fc = nn.Linear(linear_in_size, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()

    @abstractmethod
    def batch_calc_docs_embs(self, sentences):
        pass

    def forward(self, sentences, sent_lengths):
        embedded = self.batch_calc_docs_embs(sentences)
        packed_embedded = PACK(embedded, sent_lengths.cpu(), batch_first=True,
                               enforce_sorted=False)
        packed_output, (hidden_state, _) = self.lstm(packed_embedded)
        print(packed_output.data.shape)

        if self.bidirectional:
            hidden = torch.cat([hidden_state[-2,:,:], hidden_state[-1,:,:]], dim=1)
        else:
            hidden = hidden_state

        hidden = self.non_linear(hidden)
        outputs = self.fc(self.dropout(hidden))
        out = self.sigmoid(outputs)

        return out

In [17]:

class LSTMNetBert(LSTMNetBase):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def batch_calc_docs_embs(self, batch_docs):
        list_docs_embs = []
        for doc in batch_docs:
            tokenized_docs = self.tokenizer(
                doc,
                padding=True,
                truncation=True,
                return_tensors='pt',
                return_token_type_ids=False,
                return_attention_mask=False
                )

            with torch.no_grad():
                tokenized_docs = {k: v.to(nse_model.device) for k, v in tokenized_docs.items()}
                model_output = self.nse_model(**tokenized_docs)

            docs_embs = 0
            docs_embs = model_output.last_hidden_state[:, 0, :]
            docs_embs = torch.nn.functional.normalize(docs_embs)
            list_docs_embs.append(docs_embs)
        batch_docs_embs = PAD(list_docs_embs, batch_first=True)
        return batch_docs_embs

In [112]:
import torch.nn.functional as F

class RNN(nn.Module):
    """Class implementing recurrent networks (LSTM/GRU)"""
    def __init__(self, embed_size,hidden_size,num_layers=1,labels=1,
                  bidirectional=False, dropout_in=0.0,
                  dropout_out=0.0,padding_idx=0, batch_first=True,
                  LSTM=True):
        super(RNN,self).__init__()
        self.embed_size = embed_size # embedding/input dimensions
        self.hidden_size = hidden_size # hidden layers' dimensions
        self.labels = labels # output classes
        self.num_layers = num_layers # number of recurrent layers
        self.bidirectional=bidirectional # boolean: bidirectional=True makes the network bidirectional


        if LSTM:
            # If LSTM is true, use LSTM else use GRU
            self.rnn = nn.LSTM(input_size=self.embed_size,
                                hidden_size=self.hidden_size,
                                batch_first=batch_first,
                                num_layers=self.num_layers,
                                bidirectional=self.bidirectional,
                                dropout = dropout_out)

        else:
            self.rnn = nn.GRU(input_size=self.embed_size,
                                hidden_size=self.hidden_size,
                                batch_first=batch_first,
                                num_layers=self.num_layers,
                                bidirectional=self.bidirectional)

        self.dropout_in = dropout_in

        self.dropout_out = dropout_out

        if self.bidirectional:
            self.output = nn.Linear(hidden_size*2,labels)
            self.polarity = nn.Linear(hidden_size*2,2) # option to perform multitask learning with polarity of the sentence
            self.arousal = nn.Linear(hidden_size*2, 4) # option to perform multitask learning with arousal of the sentence
        else:
            self.output = nn.Linear(hidden_size, labels)
            self.polarity = nn.Linear(hidden_size,2) # option to perform multitask learning with polarity of the sentence
            self.arousal = nn.Linear(hidden_size, 4) # option to perform multitask learning with arousal of the sentence



    def forward(self, line, line_len=None, apply_softmax=False, return_final=False, classifier=False):
        """
        Parameters:
            line (tensor): the input tensor having shape [batch_size, number_of_steps, features_dimensions]
            line_len (list): a list containing the length of each sample in the batch. If no list is passed, then the function assumes all samples to have same length (i.e. no padding)
            apply_softmax (boolean): whether to apply the softmax function or not (as in the case for cross-entropy loss) after the classifier layer
            return_final (boolean): whether or not to return the final hidden state (e.g. to use it as first hidden state in a decoder)
            classifier (boolean): whether the network has a classifier layer or it acts just as an encoder"""

        if self.dropout_in:
            # if dropout_in value is not 0, then apply the dropout
            line = F.dropout(line, p=self.dropout_in)

        if line_len is not None:
            # if lengths are provided, pack the input tensor, else nothing happens
            embedded = PACK(line, line_len.data.tolist(), batch_first=True, enforce_sorted=False)
            print(embedded.data.shape)
        else:
            embedded = line

        if self.bidirectional:
            # if the network is bidirectional, first create the initial hidden and memory cell states (for LSTM)
            batch_size = line.shape[0]

            state_size = 2 * self.num_layers, batch_size, self.hidden_size


            hidden_initial = line.new_zeros(*state_size)

            cells_initial = line.new_zeros(*state_size)

            packed_out, (final_hidden_states, final_cell_states) = self.rnn(embedded,(hidden_initial,cells_initial))
            print(packed_out.data.shape)

            rnn_out, _ = PAD(packed_out, batch_first=True) # unpack the rnn output and pad it with 0s where needed
            print("unpacked out shape:", rnn_out.shape)

            if self.dropout_out:
                # if dropout_out is not 0, apply dropout to the rnn output
                rnn_out = F.dropout(rnn_out, p=self.dropout_out)

            if classifier:
                # if the network is a classifier: apply the classification layer
                rnn_out_new = rnn_out[:,-1,:].squeeze(1)
                print(rnn_out_new.shape)
                out = self.output(rnn_out_new)

            else:
                # else no output is required (the output of the network as encoder is the rnn_out)
                out = None



            if return_final:
                # if returning final hidden state, concatenate the final hidden state from forward and backward layers of the network
                def combine_directions(outs):
                    return torch.cat([outs[0: outs.size(0): 2], outs[1: outs.size(0): 2]], dim=2)
                final_hidden_states = combine_directions(final_hidden_states)
                final_cell_states = combine_directions(final_cell_states)
                return out, rnn_out, (final_hidden_states, final_cell_states)

            else:
                return out, rnn_out


        else:
            # same as the bidirectional case, but with less operations needed

            rnn_out,h_n = self.rnn(embedded)
            print(rnn_out.shape)

            if self.dropout_out:
                rnn_out = F.dropout(rnn_out, p=self.dropout_out)

            if line_len is not None:
                lengths = torch.tensor([line_len]*line.shape[0])
                rnn_out_new = column_gatherer(rnn_out, lengths)
            else:
                batch_size, seq_size, feat_size = rnn_out.shape
                rnn_out_new = rnn_out.contiguous().view(batch_size,seq_size, feat_size)[:,-1]


            if classifier:
                out = self.output(rnn_out_new)
                if apply_softmax:
                    out = F.softmax(out,dim=1)
            else:
                out = None
            if return_final:
                return out, rnn_out, h_n
            else:
                return out, rnn_out

def column_gatherer(y_out, lengths):
    """Gather the final states from a RNN with padded inputs, so that the
    actual final state for each sample can be used for classification.
    Parameters:
        y_out (tensor): the RNN output
        lengths (tensor): the individual lengths of each sample in the batch"""
    lengths = lengths.long().detach().numpy()-1
    out = []
    for batch_index, column_index in enumerate(lengths):
        out.append(y_out[batch_index, column_index])
    return torch.stack(out)

In [106]:
sample_text = """We use the Pk metric as defined in Beeferman
et al. (1999) to evaluate the performance of our
model. Pk is the probability that when passing a
sliding window of size k over sentences, the sentences at the boundaries of the window will be incorrectly classified as belonging to the same segment (or vice versa). To match the setup of Chen
et al. (2009), we also provide the Pk metric for a
sliding window over words when evaluating on the
datasets from their paper"""
sent_detector = nltk.data.load('tokenizers/punkt/russian.pickle')
sample_sents = sent_detector.tokenize(sample_text)
sample_sents = [sample_sents]
sample_lengths = [len(s) for s in sample_sents]
sample_lengths = torch.LongTensor(sample_lengths)
print(len(sample_sents))

1


In [107]:
 def batch_calc_docs_embs(batch_docs):
        list_docs_embs = []
        for doc in batch_docs:
            tokenized_docs = tokenizer(
                doc,
                padding=True,
                truncation=True,
                return_tensors='pt',
                return_token_type_ids=False,
                return_attention_mask=False
                )

            with torch.no_grad():
                tokenized_docs = {k: v.to(nse_model.device) for k, v in tokenized_docs.items()}
                model_output = nse_model(**tokenized_docs)

            docs_embs = 0
            docs_embs = model_output.last_hidden_state[:, 0, :]
            docs_embs = torch.nn.functional.normalize(docs_embs)
            list_docs_embs.append(docs_embs)
        batch_docs_embs = PAD(list_docs_embs, batch_first=True)
        return batch_docs_embs

In [113]:
sample_embs = batch_calc_docs_embs(sample_sents)

AttributeError: 'list' object has no attribute 'batch_sizes'

In [114]:
rnn = RNN(
    embed_size=312,
    hidden_size=256,
    num_layers=2,
    bidirectional=True,
)

In [115]:
out = rnn(sample_embs, sample_lengths, classifier=True)

torch.Size([5, 312])
torch.Size([5, 512])
unpacked out shape: torch.Size([1, 5, 512])
torch.Size([1, 512])


In [22]:
from transformers import AutoTokenizer, AutoModel

# SMALL SBERT
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
nse_model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

model = LSTMNetBert(
    tokenizer=tokenizer,
    nse_model=nse_model,
    hidden_dim=256,
    num_layers=2
)

model(sample_sents, sample_lengths)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


torch.Size([5, 512])


tensor([[0.4967]], grad_fn=<SigmoidBackward0>)